In [53]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import Point
import openpyxl

In [44]:
df_acidentes_raw = pd.read_csv('data/Acidentes/sinistros_com_chuva_2022-2025.csv', delimiter=';')

### Limpeza dos dados

In [45]:
# Limpeza
df_acidentes_raw['latitude'] = df_acidentes_raw['latitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['longitude'] = df_acidentes_raw['longitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['data_sinistro'] = pd.to_datetime(df_acidentes_raw['data_sinistro'], dayfirst=True)
df_acidentes_raw['hora_sinistro'] = df_acidentes_raw['hora_sinistro'].astype(str).replace('99:99', np.nan)
df_acidentes_raw['hora_sinistro_dt'] = pd.to_datetime(df_acidentes_raw['hora_sinistro'], format='%H:%M')


# Mapear 'tipo_via' para números (0: Municipal, 1: Rodovia)
tipo_via_map = {'VIAS MUNICIPAIS': 0, 'RODOVIAS': 1}
df_acidentes_raw['tipo_via_num'] = df_acidentes_raw['tipo_via'].map(tipo_via_map)

# Mapear 'Chuva' para binário (0: Sem chuva, 1: Com chuva)
df_acidentes_raw['Chuva'] = df_acidentes_raw['Chuva'].apply(lambda x: 1 if x != 'Sem chuva' else 0)

# Extrair data e hora
df_acidentes_raw['hora'] = df_acidentes_raw['hora_sinistro_dt'].dt.hour
df_acidentes_raw.fillna({'hora': -1}, inplace=True)
df_acidentes_raw['hora'] = df_acidentes_raw['hora'].astype(int)
df_acidentes_raw['data'] = df_acidentes_raw['data_sinistro'].dt.date

# Definir a classe positiva
df_acidentes_raw['Sinistro'] = 1

colunas_veiculos = [col for col in df_acidentes_raw.columns if 'tp_veiculo' in col]
colunas_base = ['latitude', 'longitude', 'data', 'hora', 'Sinistro', 'Chuva', 'tipo_via_num']
df_positivos = df_acidentes_raw[colunas_base + colunas_veiculos]

print(f"Total de positivos: {len(df_positivos)}")
df_positivos

Total de positivos: 8189


,latitude,longitude,data,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,2022-01-01,4,1,0,0.0,0,0,0,0,0,0,0
1,-22.305238,-49.097946,2022-01-01,4,1,0,0.0,0,0,0,0,0,0,0
2,-22.306460,-49.104453,2022-01-01,4,1,0,0.0,0,0,1,0,0,0,0
3,-22.312349,-49.122265,2022-01-02,19,1,0,0.0,0,0,0,0,0,0,0
4,-22.338166,-49.053244,2022-01-02,20,1,0,0.0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.391376,-49.069106,2025-02-28,22,1,0,0.0,0,0,0,0,0,0,3
8185,-22.295708,-49.045586,2025-02-28,13,1,0,0.0,0,0,1,0,0,0,1
8186,-22.338551,-49.069617,2025-02-28,13,1,0,0.0,0,0,0,0,0,0,2
8187,-22.305703,-49.051948,2025-02-28,18,1,0,0.0,0,0,0,0,0,0,1


### Geração dos Negativos Tipo 1 (Mesmo Local; Data, Hora e Chuva Aleatório)

In [46]:
# O lookup não muda, pois a chave de um evento continua sendo (local, tempo)
lookup_acidentes = set(zip(
    df_positivos['latitude'],
    df_positivos['longitude'],
    df_positivos['data'],
    df_positivos['hora']
))

# --- Funções de Geração Aleatória ---
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 2, 28)
total_days = (end_date - start_date).days

def gerar_timestamp_aleatorio():
    data_aleatoria = start_date + timedelta(days=random.randint(0, total_days))
    hora_aleatoria = random.randint(0, 23)
    return data_aleatoria.date(), hora_aleatoria

# Sorteia chuva com uma probabilidade (ex: 20% de chance de chover)
def sortear_chuva(probabilidade_chuva=0.2):
    return 1 if random.random() < probabilidade_chuva else 0

# --- Geração ---
negativos_tipo1_lista = []
total_positivos = len(df_positivos)
prop_neg = 1

for index, acidente in df_positivos.iterrows():
    
    lat = acidente['latitude']
    lon = acidente['longitude']
    
    for i in range(prop_neg):

        while True:
            data_nova, hora_nova = gerar_timestamp_aleatorio()
            if (lat, lon, data_nova, hora_nova) not in lookup_acidentes:
                break
            
    # Criando o dicionário do novo negativo
    novo_negativo = {
        'latitude': lat,
        'longitude': lon,
        'data': data_nova,
        'hora': hora_nova,
        'Sinistro': 0,
        'Chuva': sortear_chuva(),
        'tipo_via_num': acidente['tipo_via_num'] # Mantém o tipo de via do local
    }
    # Adiciona colunas de veículo com valor 0
    for col in colunas_veiculos:
        novo_negativo[col] = 0
        
    negativos_tipo1_lista.append(novo_negativo)

df_negativos_tipo1 = pd.DataFrame(negativos_tipo1_lista)
df_negativos_tipo1

,latitude,longitude,data,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,2022-12-26,23,0,0,0.0,0,0,0,0,0,0,0
1,-22.305238,-49.097946,2022-01-13,22,0,1,0.0,0,0,0,0,0,0,0
2,-22.306460,-49.104453,2022-08-28,9,0,0,0.0,0,0,0,0,0,0,0
3,-22.312349,-49.122265,2023-12-22,1,0,0,0.0,0,0,0,0,0,0,0
4,-22.338166,-49.053244,2024-02-24,20,0,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.391376,-49.069106,2022-03-04,3,0,0,0.0,0,0,0,0,0,0,0
8185,-22.295708,-49.045586,2023-07-09,11,0,0,0.0,0,0,0,0,0,0,0
8186,-22.338551,-49.069617,2024-10-18,5,0,0,0.0,0,0,0,0,0,0,0
8187,-22.305703,-49.051948,2024-10-22,6,0,0,0.0,0,0,0,0,0,0,0


### Geração dos Negativos Tipo 2 (Local Aleatório; Data, Hora e tempo Aleatório)

In [49]:
gdf_ruas = gpd.read_file('ruas_de_bauru.gpkg')

# Mapeia os tipos de via do OpenStreetMap para o seu padrão (0 ou 1)
highway_map = {
     'motorway': 1, 'trunk': 1, 'primary': 0, 'secondary': 0,
     'tertiary': 0, 'residential': 0, 'unclassified': 0
}

gdf_ruas['tipo_via_num'] = gdf_ruas['highway'].map(highway_map).fillna(0)

min_lon, min_lat, max_lon, max_lat = gdf_ruas.total_bounds # Delimita Bauru
uniao_ruas = gdf_ruas.union_all().buffer(0.0001)

negativos_tipo2_lista = []
num_negativos_tipo2 = len(df_positivos) * 1  # Proporção 1:1

while len(negativos_tipo2_lista) < num_negativos_tipo2:
    lon_aleatorio = random.uniform(min_lon, max_lon)
    lat_aleatoria = random.uniform(min_lat, max_lat)
    ponto_aleatorio = Point(lon_aleatorio, lat_aleatoria)

    if ponto_aleatorio.within(uniao_ruas):
        # Encontra a rua mais próxima do ponto para extrair o tipo de via
        rua_mais_proxima = gdf_ruas.iloc[gdf_ruas.sindex.nearest(ponto_aleatorio)[1]]
        tipo_via_ponto = rua_mais_proxima['tipo_via_num'].iloc[0]
    
        data_aleatoria, hora_aleatoria = gerar_timestamp_aleatorio()
    
        novo_negativo = {
            'latitude': lat_aleatoria,
            'longitude': lon_aleatorio,
            'data': data_aleatoria,
            'hora': hora_aleatoria,
            'Sinistro': 0,
            'Chuva': sortear_chuva(),
            'tipo_via_num': tipo_via_ponto
        }

        for col in colunas_veiculos:
            novo_negativo[col] = 0

        negativos_tipo2_lista.append(novo_negativo)

        if len(negativos_tipo2_lista) % 500 == 0:
            print(f"Gerado {len(negativos_tipo2_lista)}/{num_negativos_tipo2} negativos tipo 2...")

df_negativos_tipo2 = pd.DataFrame(negativos_tipo2_lista)
df_negativos_tipo2

Gerado 500/8189 negativos tipo 2...
Gerado 1000/8189 negativos tipo 2...
Gerado 1500/8189 negativos tipo 2...
Gerado 2000/8189 negativos tipo 2...
Gerado 2500/8189 negativos tipo 2...
Gerado 3000/8189 negativos tipo 2...
Gerado 3500/8189 negativos tipo 2...
Gerado 4000/8189 negativos tipo 2...
Gerado 4500/8189 negativos tipo 2...
Gerado 5000/8189 negativos tipo 2...
Gerado 5500/8189 negativos tipo 2...
Gerado 6000/8189 negativos tipo 2...
Gerado 6500/8189 negativos tipo 2...
Gerado 7000/8189 negativos tipo 2...
Gerado 7500/8189 negativos tipo 2...
Gerado 8000/8189 negativos tipo 2...


,latitude,longitude,data,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.332152,-49.018277,2023-03-26,20,0,0,0.0,0,0,0,0,0,0,0
1,-22.299891,-49.048772,2022-05-25,7,0,0,0.0,0,0,0,0,0,0,0
2,-22.333140,-49.015139,2022-03-18,20,0,0,1.0,0,0,0,0,0,0,0
3,-22.277774,-49.063259,2023-08-31,8,0,0,0.0,0,0,0,0,0,0,0
4,-22.343782,-49.063791,2024-12-12,14,0,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.255688,-49.070251,2024-06-15,17,0,0,0.0,0,0,0,0,0,0,0
8185,-22.356731,-49.050073,2022-03-21,8,0,0,0.0,0,0,0,0,0,0,0
8186,-22.248434,-49.055627,2023-02-08,12,0,0,0.0,0,0,0,0,0,0,0
8187,-22.306864,-49.088638,2024-08-20,4,0,0,0.0,0,0,0,0,0,0,0


### Unir, embaralhar e salvar

In [57]:
df_final = pd.concat([df_positivos, df_negativos_tipo1], ignore_index=True)
df_final = pd.concat([df_final, df_negativos_tipo2], ignore_index=True)

# Embaralha o dataset final para garantir aleatoriedade
df_final = df_final.sample(frac=1).reset_index(drop=True)

df_final.to_csv('dataset_final_para_modelo.csv', index=False, decimal=',')

print(f"Total de amostras: {len(df_final)}")
print("Distribuição das classes:")
print(df_final['Sinistro'].value_counts())
df_final

Total de amostras: 24567
Distribuição das classes:
Sinistro
0    16378
1     8189
Name: count, dtype: int64


,latitude,longitude,data,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.343254,-49.077116,2023-12-03,10,0,0,0.0,0,0,0,0,0,0,0
1,-22.320025,-49.068560,2022-02-26,9,1,0,0.0,0,0,0,0,0,0,2
2,-22.316968,-48.994278,2024-09-12,10,0,0,0.0,0,0,0,0,0,0,0
3,-22.303559,-49.057237,2022-05-08,13,1,0,0.0,0,0,0,0,0,0,0
4,-22.332952,-49.028316,2023-08-13,13,1,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24562,-22.246952,-49.080833,2023-08-11,18,0,1,0.0,0,0,0,0,0,0,0
24563,-22.334902,-49.088110,2023-08-15,16,0,0,0.0,0,0,0,0,0,0,0
24564,-22.342904,-49.115687,2022-03-17,17,1,0,0.0,0,0,0,0,0,0,0
24565,-22.311962,-49.070028,2023-10-24,10,1,0,0.0,0,0,0,0,0,0,0
